In [ ]:
pip install openmeteo-requests

In [ ]:
pip install requests-cache retry-requests numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
# Creando la conexión a Redsshift
import psycopg2
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="fedebaer_coderhouse"
with open("C:\Fede\Data Engineer Coderhouse\pwd_redshift_fd.txt",'r') as f:
    pwd= f.read()
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")

except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [ ]:
#Crear la tabla en redshift si no existe
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS fedebaer_coderhouse.daily_weather(
        m_date datetime null,
        weather_code float null,
        temperature_2m_max float null,
        temperature_2m_min float null,
        apparent_temperature_max float null,
        apparent_temperature_min float null,
        sunrise float null,
        sunset float null,
        daylight_duration float null,
        sunshine_duration float null,
        precipitation_sum float null,
        rain_sum float null,
        showers_sum float null,
        snowfall_sum float null,
        precipitation_hours float null,
        precipitation_probability_max float null
        )
    """)
    conn.commit()

In [ ]:
# Conexión con la API de clima
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# weather variables list
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": [-32.8908],
	"longitude": [-68.8272],
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_hours", "precipitation_probability_max"],
	"timezone": ["America/Sao_Paulo"],
	"past_days": 92,
	"elevation": ["NaN"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
daily_sunrise = daily.Variables(5).ValuesAsNumpy()
daily_sunset = daily.Variables(6).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(7).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(8).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(9).ValuesAsNumpy()
daily_rain_sum = daily.Variables(10).ValuesAsNumpy()
daily_showers_sum = daily.Variables(11).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(12).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(13).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(14).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["showers_sum"] = daily_showers_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates -32.875°N -68.875°E
Elevation 1188.0 m asl
Timezone b'America/Sao_Paulo' b'-03'
Timezone difference to GMT+0 -10800 s
                        date  weather_code  temperature_2m_max  \
0  2023-12-31 03:00:00+00:00          80.0           27.450001   
1  2024-01-01 03:00:00+00:00           3.0           28.799999   
2  2024-01-02 03:00:00+00:00          80.0           28.549999   
3  2024-01-03 03:00:00+00:00          80.0           28.549999   
4  2024-01-04 03:00:00+00:00          80.0           31.000000   
..                       ...           ...                 ...   
94 2024-04-03 03:00:00+00:00           2.0           18.700001   
95 2024-04-04 03:00:00+00:00           1.0           21.450001   
96 2024-04-05 03:00:00+00:00           1.0           21.650000   
97 2024-04-06 03:00:00+00:00           0.0           23.200001   
98 2024-04-07 03:00:00+00:00           3.0           25.750000   

    temperature_2m_min  apparent_temperature_max  apparent_temperature_min  \

In [ ]:
#Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO fedebaer_coderhouse.daily_weather (
    m_date,
    weather_code,
    temperature_2m_max,
    temperature_2m_min,
    apparent_temperature_max,
    apparent_temperature_min,
    sunrise,
    sunset,
    daylight_duration,
    sunshine_duration,
    precipitation_sum,
    rain_sum,
    showers_sum,
    snowfall_sum,
    precipitation_hours,
    precipitation_probability_max)
    VALUES %s
        ''',
        [tuple(row) for row in daily_dataframe.values],
        page_size=len(daily_dataframe)
    )
    conn.commit()